# Motion Planning Examples

In [ ]:
from robodk import robolink
from path_planning.rdk_path_planning import *


# connect to the RoboDK API
rdk = robolink.Robolink()

# define the weights for the distance function
decay_rate = 0.45
weights = np.exp(-np.arange(6)*decay_rate)
# the distance function allows us to calculate the cost of moving between two robot configurations
# the cost is calculated as the weighted sum of the absolute difference between the joint values
dist_function = lambda J1,J2: eucledean_distance(J1,J2,weights)

# define the start and goal configurations (stored as a list of joint values)
J1 = rdk.Item("J1").Joints().tolist()
J2 = rdk.Item("J2").Joints().tolist()

# Probabilistic Roadmap Motion Planning

Here we show the ability to load different PRM maps and use them to plan trajectories between J1 and J2

This expands on the current capabilities of RoboDK by letting you load multiple PRM maps. These different maps could each correspond to a known configuration of the robot cell (example: door open vs door closed)

In [20]:
collision_map = CollisionMap.load("PRM_1.npz")
rdk_path_planner = RDKPathPlanner(
    rdk = rdk, robot_name='Omron TM12X',min_step=5.0,configuration_distance_func=dist_function, collision_map=collision_map)
J1 = rdk.Item("J1").Joints().tolist()
J2 = rdk.Item("J2").Joints().tolist()
path = rdk_path_planner.plan_path(J1,J2)
if path:
    rdk_path_planner.robot.setJoints(path[0])
for j in path:
    rdk_path_planner.robot.MoveJ(j)


[170]


In [19]:
collision_map = CollisionMap.load("PRM_2.npz")
rdk_path_planner = RDKPathPlanner(
    rdk = rdk, robot_name='Omron TM12X',min_step=5.0,configuration_distance_func=dist_function, collision_map=collision_map)
J1 = rdk.Item("J1").Joints().tolist()
J2 = rdk.Item("J2").Joints().tolist()
path = rdk_path_planner.plan_path(J1,J2)
if path:
    rdk_path_planner.robot.setJoints(path[0])
for j in path:
    rdk_path_planner.robot.MoveJ(j)

[456, 43, 75, 450]


# Creating a new PRM and saving it to a file

In [ ]:
rdk = robolink.Robolink()
rdk_path_planner = RDKPathPlanner(
    rdk = rdk, robot_name='Omron TM12X',
    min_step=5.0,configuration_distance_func=dist_function)
rdk_path_planner.generate_prm(n_samples=500,max_connections_per_node=5,neighborhood_radius=100)
rdk_path_planner.collision_map.save("PRM_3.npz")

# Rapidly Exploring Random Trees (RRT)

If the robot station can have configuration states that cannot be known ahead of time, it is impossible to have a set of PRM maps that would work for every situation. The RRT approach allows us to create a tree structure which starts at J1 and tries to "grow" towards J2. The resultant motion is a bit zig-zaggy, but it is considerably faster in generating a plan when a PRM approach is not feasable.

In [23]:
# create the path planner object
rdk_path_planner = RDKPathPlanner(rdk = rdk, robot_name='Omron TM12X',min_step=5.0,configuration_distance_func=dist_function)
# use the RRT algorithm to generate a path between the start and goal configurations
# this will return a list of joint configurations. an empty list means that no path was found
plan = rdk_path_planner.generate_rrt_plan(J1,J2,max_iterations=200,goal_bias=0.1,step_size=20.0)
# if a plan is found, move the robot along the path
if plan:
    rdk_path_planner.robot.setJoints(plan[0])
    for j in plan:
        rdk_path_planner.robot.MoveJ(j)

 45%|████▌     | 90/200 [00:13<00:16,  6.51it/s]


Solution found
